In [1]:
!pip install surprise

     |████████████████████████████████| 11.8MB 5.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670908 sha256=8c5f752a0ed5f72ecfec9da7812347f1175d93cb2172b102b935985ffefd1270
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import surprise
from surprise import SVD
from surprise import accuracy
from surprise import KNNBasic, KNNWithMeans, KNNBaseline
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import  Dataset
from surprise.model_selection import cross_validate,GridSearchCV
import re
import random 
import requests

In [3]:
movie_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/movie.csv')
rating_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/rating.csv')
link_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/link.csv')
tag_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/tag.csv')
genome_tags_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/genome_tags.csv')
genome_scores_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/genome_scores.csv')

In [4]:
combined_small = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movie_lens/combined_small_v2.csv')

In [5]:
model = surprise.dump.load('/content/drive/MyDrive/Colab Notebooks/movie_lens/my_model_v3')
predictions = model[0]

In [6]:
# check the accuracy using Root Mean Square Error
accuracy.rmse(predictions)

RMSE: 0.9801


0.9800557814867045

In [7]:
def string_preprocess(row):
  text = row
  text = str(text)
  text = re.sub(r'\(\d+\)', '', text)
  text = text.lower()
  return text

In [16]:
#movie_rating cropped
movie_rating_combined = pd.merge(movie_df,rating_df,on='movieId')
movie_info_small = movie_rating_combined

In [21]:
movie_info_small

,movieId,title,genres,userId,rating,timestamp
0,1,toy story,Adventure|Animation|Children|Comedy|Fantasy,3,4.0,1999-12-11 13:36:47
1,1,toy story,Adventure|Animation|Children|Comedy|Fantasy,6,5.0,1997-03-13 17:50:52
2,1,toy story,Adventure|Animation|Children|Comedy|Fantasy,8,4.0,1996-06-05 13:37:51
3,1,toy story,Adventure|Animation|Children|Comedy|Fantasy,10,4.0,1999-11-25 02:44:47
4,1,toy story,Adventure|Animation|Children|Comedy|Fantasy,11,4.5,2009-01-02 01:13:41
...,...,...,...,...,...,...
20000258,131254,kein bund für's leben,Comedy,79570,4.0,2015-03-30 19:32:59
20000259,131256,"feuer, eis & dosenbier",Comedy,79570,4.0,2015-03-30 19:48:08
20000260,131258,the pirates,Adventure,28906,2.5,2015-03-30 19:56:32
20000261,131260,rentun ruusu,(no genres listed),65409,3.0,2015-03-30 19:57:46


In [39]:
id = movie_info_small['movieId'].unique().tolist()
name = movie_info_small['title'].unique().tolist()

In [18]:
movie_info_small['title'] = movie_info_small['title'].apply(string_preprocess)

In [38]:
#create index item name and value item id series
movies_list_id = pd.Series(movie_info_small['movieId'].unique().tolist(),index=movie_info_small['title'].unique().tolist())

ValueError: ignored

In [42]:
movies_list_id.tail()

philadelphia story, the     898
singin' in the rain         899
american in paris, an       900
funny face                  901
breakfast at tiffany's      902
dtype: int64

In [13]:
def get_movies_recommendation(movie_name):
  movie_name = movie_name.lower()
  item_id = []
  rating_id = []
  movie_id = movies_list_id.loc[movies_list_id.index.str.contains(movie_name)][0]
  user_id = combined_small.loc[combined_small.item == movie_id]['user'].tolist()[:5]
  for id in user_id:
    for i in range(len(predictions)):
      if predictions[i].uid == id:
        item_id.append(predictions[i].iid)
        rating_id.append((predictions[i].est,predictions[i].iid))
  movies_names = movies_list_id.loc[movies_list_id.isin(item_id)].index.tolist()[:5]
  movies_ratings = dict(zip(movies_names,rating_id))
  return movies_ratings

In [14]:
#function to make api call to tmdb
def tmdb_call(movie_name):
  movies = get_movies_recommendation(movie_name)
  movie_ids = []
  datas = []

  #getting the ids of the movies
  for key,value in movies.items():
    movie_ids.append(value[1])

  #collecting the tmdb id for link_df
  tmdb_ids = link_df.loc[link_df.movieId.isin(keys)]['tmdbId'].values.tolist()
  for id in tmdb_ids:
    url = f"https://api.themoviedb.org/3/movie/{id}?api_key=f86248c492db78432f0f03f7246698d6"
    resp = requests.get(url=url)
    data = resp.json()
    datas.append(data)
  
  return datas

In [43]:
get_movies_recommendation('virtuosity')

{'american president, the ': (3.4481593123722303, 338),
 'father of the bride part ii ': (2.6673910080539946, 173),
 'leaving las vegas ': (4.396927752929507, 364),
 'to die for ': (2.7323745259997603, 588),
 'usual suspects, the ': (3.8030206261310884, 289)}